# Workshop Microsoft - Sostenibilidad
En el siguiente ejemplo analizaremos el coste energético de un modelo de la inferencia del modelo Phi de Microsoft.
Se puede realizar este ejemplo con CPU o GPU. En caso de ser con GPU es necesario indicarlo en los parámetros de configuración

In [33]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import textwrap
from codecarbon import EmissionsTracker
from codecarbon import track_emissions
import pandas as pd
import os
import torch
from dotenv import load_dotenv

load_dotenv()

False

In [25]:
# parametros de configuración
model_name = "microsoft/Phi-3.5-mini-instruct"
output_dir_emissions_track="/home/azureuser/cloudfiles/code/Users/paul.vanb/worshop-ia-sostenible/outputs"
device="cuda"

In [26]:

def wrap_text(text, width=100):
    """
    Envuelve el texto dado para que cada línea no supere el ancho especificado.
    
    Args:
        text (str): El texto de entrada.
        width (int): El ancho máximo de cada línea (por defecto, 100 caracteres).
    
    Returns:
        str: Texto con saltos de línea insertados para cumplir con el ancho especificado.
    """
    lines = text.split('\n')  # Divide el texto en líneas basadas en saltos de línea existentes.
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]  # Envuelve cada línea individualmente.
    wrapped_text = '\n'.join(wrapped_lines)  # Une las líneas envueltas con saltos de línea.
    return wrapped_text

def generate(input_text, system_prompt="", max_length=1024):
    """
    Genera una respuesta basada en el texto de entrada utilizando un modelo de IA.
    
    Args:
        input_text (str): Texto de entrada del usuario.
        system_prompt (str): Mensaje de sistema opcional para guiar la respuesta del modelo.
        max_length (int): Longitud máxima de la respuesta generada (por defecto, 1024 caracteres).
    
    Returns:
        str: Texto generado y formateado con saltos de línea adecuados.
    """
    # Si no se proporciona un mensaje del sistema, se usa un mensaje por defecto.
    if system_prompt != "":
        system_prompt = system_prompt
    else:
        system_prompt = "You are a friendly and helpful assistant"
    
    # Construcción del historial de mensajes con el mensaje del sistema y la entrada del usuario.
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": input_text},
    ]
    
    # Generación de la respuesta del modelo.
    output = pipe(messages, **generation_args)
    text = output[0]['generated_text']  # Obtiene el texto generado.
    
    # Envuelve el texto generado para cumplir con el ancho especificado.
    wrapped_text = wrap_text(text)
    return wrapped_text

@track_emissions(project_name="workshop-sostenibilidad", output_dir=output_dir_emissions_track,
                 cloud_provider="azure", cloud_region="North Europe", pue=1.19)
def answer_to_different_messages(messages):
    """
    Procesa una lista de mensajes y genera respuestas mientras realiza un seguimiento de las emisiones.
    
    Args:
        messages (list): Lista de mensajes de entrada.
    
    Returns:
        list: Lista de respuestas generadas junto con los mensajes de entrada y sus identificadores.
    """
    outputs = []  # Lista para almacenar los resultados.
    system_prompt = "Eres un asistente de Microsoft experto en sostenibilidad llamado SostAI"  # Mensaje del sistema.
    
    # Itera sobre cada mensaje y genera una respuesta.
    for mid, message in enumerate(messages):
        output = generate(message, system_prompt=system_prompt)  # Genera la respuesta.
        outputs.append([mid, message, output])  # Almacena el identificador, mensaje y respuesta.
    
    return outputs  # Devuelve la lista de respuestas generadas.

In [30]:
df_emissions = pd.read_csv(os.path.join(output_dir_emissions_track,os.path.join(output_dir_emissions_track,"emissions.csv")))
df_emissions.head()

,timestamp,project_name,run_id,experiment_id,duration,emissions,emissions_rate,cpu_power,gpu_power,ram_power,...,cpu_count,cpu_model,gpu_count,gpu_model,longitude,latitude,ram_total_size,tracking_mode,on_cloud,pue
0,2025-02-19T12:02:50,codecarbon,3bfd136c-d8e5-4f4e-a98e-a51239239023,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,17.135990,0.000041,0.000002,97.5,0.0,3.091317,...,4,Intel(R) Xeon(R) Platinum 8272CL CPU @ 2.60GHz,NaN,NaN,-122.3303,47.6109,31.343273,process,N,1.0
1,2025-02-19T12:03:10,codecarbon,9bc68c95-44f7-4838-9b5e-00de148b7f57,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,17.254767,0.000044,0.000003,97.5,0.0,11.753727,...,4,Intel(R) Xeon(R) Platinum 8272CL CPU @ 2.60GHz,NaN,NaN,-122.3303,47.6109,31.343273,machine,N,1.0
2,2025-02-19T12:04:54,workshop-sostenibilidad,ef6ae2a3-bd22-469f-aa2e-e12b329f5886,5b0fa12a-3dd7-45bb-9766-cc326314d9f1,63.112533,0.000162,0.000003,97.5,0.0,11.753727,...,4,Intel(R) Xeon(R) Platinum 8272CL CPU @ 2.60GHz,NaN,NaN,-122.3303,47.6109,31.343273,machine,N,1.0
3,2025-02-19T15:49:45,codecarbon,c29eaeff-7e73-4de2-81f5-2d7d7e121a0f,D08fa668-aed9-4ee9-a535-dedb1361039c,20.548237,0.000049,0.000002,97.5,0.0,3.047070,...,4,Intel(R) Xeon(R) Platinum 8272CL CPU @ 2.60GHz,NaN,NaN,-122.3303,47.6109,31.343277,process,N,1.0
4,2025-02-23T09:48:15,codecarbon,d50c6d8e-a5bd-46a7-8b1d-18e21a2c39b1,D08fa668-aed9-4ee9-a535-dedb1361039c,29.065790,0.000069,0.000002,97.5,0.0,3.061984,...,4,Intel(R) Xeon(R) Platinum 8272CL CPU @ 2.60GHz,NaN,NaN,-122.3303,47.6109,31.343277,process,N,1.0


# Uso del dashboard

In [29]:
#carbonboard --filepath=os.path.join(output_dir_emissions_track,"emissions.csv") --port=3333

# Uso de la API

https://dashboard.codecarbon.io/

In [ ]:
# en local
#!codecarbon login
#!codecarbon config

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/works-sost-cpu/code'

True

In [18]:
from dotenv import load_dotenv

load_dotenv()

False

In [19]:
CODECARBON_API_KEY = os.environ["CODECARBON_API_KEY"]
CODECARBON_EXPERIMENT_ID = os.environ["CODECARBON_EXPERIMENT_ID"]

In [ ]:

%%time
# Start tracking
tracker = EmissionsTracker(tracking_mode="process", save_to_api=True,api_key= CODECARBON_API_KEY,
                           output_dir=output_dir_emissions_track,
                 )
tracker.start()

generate('¿Quién eres?',
         system_prompt="Eres un asistente majo",
         )
emmissions = tracker.stop()

[codecarbon INFO @ 17:39:09] Codecarbon is taking the configuration from global file: /home/azureuser/.codecarbon.config
[codecarbon INFO @ 17:39:10] [setup] RAM Tracking...
[codecarbon INFO @ 17:39:10] [setup] CPU Tracking...
[codecarbon WARNING @ 17:39:10] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU

[codecarbon INFO @ 17:39:11] CPU Model on constant consumption mode: Intel(R) Xeon(R) Platinum 8272CL CPU @ 2.60GHz
[codecarbon INFO @ 17:39:11] [setup] GPU Tracking...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has 

<class 'str'>


[codecarbon ERROR @ 17:39:12] ApiClient Error when calling the API on https://api.codecarbon.io/runs with : {"timestamp": "2025-02-23T17:39:11.687535+00:00", "experiment_id": "D08fa668-aed9-4ee9-a535-dedb1361039c", "os": "Linux-5.15.0-1073-azure-x86_64-with-glibc2.31", "python_version": "3.10.14", "codecarbon_version": "2.8.3", "cpu_count": 4, "cpu_model": "Intel(R) Xeon(R) Platinum 8272CL CPU @ 2.60GHz", "gpu_count": null, "gpu_model": null, "longitude": -122.3, "latitude": 47.6, "region": null, "provider": null, "ram_total_size": 31.343276977539062, "tracking_mode": "process"}
[codecarbon ERROR @ 17:39:12] ApiClient API return http code 403 and answer : {"detail":"Not allowed to perform this action"}
[codecarbon INFO @ 17:39:24] Energy consumed for RAM : 0.000077 kWh. RAM Power : 3.093413829803467 W
[codecarbon INFO @ 17:39:24] Energy consumed for all CPUs : 0.002438 kWh. Total CPU Power : 97.5 W
[codecarbon INFO @ 17:39:24] 0.002515 kWh of electricity used since the beginning.
[code

<class 'str'>


[codecarbon ERROR @ 17:39:37] ApiClient Error when calling the API on https://api.codecarbon.io/runs with : {"timestamp": "2025-02-23T17:39:37.091444+00:00", "experiment_id": "D08fa668-aed9-4ee9-a535-dedb1361039c", "os": "Linux-5.15.0-1073-azure-x86_64-with-glibc2.31", "python_version": "3.10.14", "codecarbon_version": "2.8.3", "cpu_count": 4, "cpu_model": "Intel(R) Xeon(R) Platinum 8272CL CPU @ 2.60GHz", "gpu_count": null, "gpu_model": null, "longitude": -122.3, "latitude": 47.6, "region": null, "provider": null, "ram_total_size": 31.343276977539062, "tracking_mode": "process"}
[codecarbon ERROR @ 17:39:37] ApiClient API return http code 403 and answer : {"detail":"Not allowed to perform this action"}
[codecarbon ERROR @ 17:39:37] ApiClient.add_emission still no run_id, aborting for this time !


...

[codecarbon ERROR @ 17:43:54] ApiClient Error when calling the API on https://api.codecarbon.io/runs with : {"timestamp": "2025-02-23T17:43:54.264744+00:00", "experiment_id": "D08fa668-aed9-4ee9-a535-dedb1361039c", "os": "Linux-5.15.0-1073-azure-x86_64-with-glibc2.31", "python_version": "3.10.14", "codecarbon_version": "2.8.3", "cpu_count": 4, "cpu_model": "Intel(R) Xeon(R) Platinum 8272CL CPU @ 2.60GHz", "gpu_count": null, "gpu_model": null, "longitude": -122.3, "latitude": 47.6, "region": null, "provider": null, "ram_total_size": 31.343276977539062, "tracking_mode": "process"}
[codecarbon ERROR @ 17:43:54] ApiClient API return http code 403 and answer : {"detail":"Not allowed to perform this action"}
[codecarbon ERROR @ 17:43:54] ApiClient.add_emission still no run_id, aborting for this time !
[codecarbon INFO @ 17:44:09] Energy consumed for RAM : 0.000322 kWh. RAM Power : 3.0967140197753906 W
[codecarbon INFO @ 17:44:09] Energy consumed for all CPUs : 0.010156 kWh. Total CPU Power :

<class 'str'>


[codecarbon ERROR @ 17:45:54] ApiClient Error when calling the API on https://api.codecarbon.io/runs with : {"timestamp": "2025-02-23T17:45:54.267778+00:00", "experiment_id": "D08fa668-aed9-4ee9-a535-dedb1361039c", "os": "Linux-5.15.0-1073-azure-x86_64-with-glibc2.31", "python_version": "3.10.14", "codecarbon_version": "2.8.3", "cpu_count": 4, "cpu_model": "Intel(R) Xeon(R) Platinum 8272CL CPU @ 2.60GHz", "gpu_count": null, "gpu_model": null, "longitude": -122.3, "latitude": 47.6, "region": null, "provider": null, "ram_total_size": 31.343276977539062, "tracking_mode": "process"}
[codecarbon ERROR @ 17:45:54] ApiClient API return http code 403 and answer : {"detail":"Not allowed to perform this action"}
[codecarbon ERROR @ 17:45:54] ApiClient.add_emission still no run_id, aborting for this time !
[codecarbon INFO @ 17:46:09] Energy consumed for RAM : 0.000425 kWh. RAM Power : 3.096892833709717 W
[codecarbon INFO @ 17:46:09] Energy consumed for all CPUs : 0.013406 kWh. Total CPU Power : 

<class 'str'>


[codecarbon ERROR @ 17:47:55] ApiClient Error when calling the API on https://api.codecarbon.io/runs with : {"timestamp": "2025-02-23T17:47:54.270815+00:00", "experiment_id": "D08fa668-aed9-4ee9-a535-dedb1361039c", "os": "Linux-5.15.0-1073-azure-x86_64-with-glibc2.31", "python_version": "3.10.14", "codecarbon_version": "2.8.3", "cpu_count": 4, "cpu_model": "Intel(R) Xeon(R) Platinum 8272CL CPU @ 2.60GHz", "gpu_count": null, "gpu_model": null, "longitude": -122.3, "latitude": 47.6, "region": null, "provider": null, "ram_total_size": 31.343276977539062, "tracking_mode": "process"}
[codecarbon ERROR @ 17:47:55] ApiClient API return http code 403 and answer : {"detail":"Not allowed to perform this action"}
[codecarbon ERROR @ 17:47:55] ApiClient.add_emission still no run_id, aborting for this time !
[codecarbon INFO @ 17:48:09] Energy consumed for RAM : 0.000528 kWh. RAM Power : 3.097469329833985 W
[codecarbon INFO @ 17:48:09] Energy consumed for all CPUs : 0.016656 kWh. Total CPU Power : 